In [80]:
import torch
from transformers import VitsModel, AutoTokenizer
import numpy as np
from pydub import AudioSegment
import os
import uuid
import scipy.io.wavfile
from pydub import AudioSegment
import time  # Добавляем импорт модуля time
# AudioSegment.converter = "/path/to/ffmpeg"



In [81]:


class TextToSpeechSynthesizer:
    """
    A synthesizer for converting text to speech using a VITS model.
    It outputs the speech in MP3 format.
    """
    def __init__(self, model_name="facebook/mms-tts-kir", output_dir="./media/voices"):
        self.model = VitsModel.from_pretrained(model_name)  
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.output_dir = output_dir
        os.makedirs(self.output_dir, exist_ok=True)

    def synthesize(self, text):
        inputs = self.tokenizer(text, return_tensors="pt")
        inputs['input_ids'] = inputs['input_ids'].long()

        if inputs['input_ids'].size(1) == 0:
            raise ValueError("Tokenized input is empty. Check the input text or tokenizer settings.")
        
        try:
            with torch.no_grad():
                outputs = self.model(**inputs)
        except RuntimeError as e:
            print("Error during model inference:", e)
            raise

        waveform = outputs.waveform.squeeze().cpu().numpy()
        sampling_rate = self.model.config.sampling_rate

        uid = str(uuid.uuid4())
        mp3_path = os.path.join(self.output_dir, f"{uid}.mp3")
        scipy.io.wavfile.write(mp3_path, rate=sampling_rate, data=waveform)
        time.sleep(3)

        return mp3_path


    def kyrgyz_number(n):
        if n < 10:
            return number_words[str(n)]
        elif n < 100:
            tens, below_ten = divmod(n, 10)
            if below_ten == 0:
                return number_words[str(tens * 10)]
            else:
                return number_words[str(tens * 10)] + " " + number_words[str(below_ten)]
        elif n < 1000:
            hundreds, below_hundred = divmod(n, 100)
            result = number_words[str(hundreds)] + ' ' + number_words['100'] if hundreds > 1 else number_words['100']
            if below_hundred != 0:
                result += " " + kyrgyz_number(below_hundred)
            return result
        elif n < 10000:
            thousands, below_thousand = divmod(n, 1000)
            result = number_words[str(thousands)] + ' ' + number_words['1000'] if thousands > 1 else number_words['1000']
            if below_thousand != 0:
                result += " " + kyrgyz_number(below_thousand)
            return result
        else:
            return str(n)  

    global number_words = {
        '0': 'ноль', '1': 'бир', '2': 'эки', '3': 'уч', '4': 'төрт', '5': 'беш', '6': 'алты',
        '7': 'жети', '8': 'сегиз', '9': 'тогуз', '10': 'он', '20': 'жыйырма', '30': 'отуз',
        '40': 'кырк', '50': 'элу', '60': 'алтымыш', '70': 'жетимиш', '80': 'сексен',
        '90': 'токсон', '100': 'жүз', '1000': 'миң'
    }


if __name__ == "__main__":
    output_directory = "media/voices"
    synthesizer = TextToSpeechSynthesizer(output_dir=output_directory)
    example_text = "   " + " 500 миң сап ырды түзөт."
    example_text = synthesizer.num_to_text(example_text.strip())
    try:
        mp3_file = synthesizer.synthesize(example_text)
        print(f"Generated speech saved to {mp3_file}")
    except Exception as e:
        print(f"Error: {e}")


Some weights of the model checkpoint at facebook/mms-tts-kir were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

KeyError: '500'

In [86]:
import torch
from transformers import VitsModel, AutoTokenizer
import numpy as np
from pydub import AudioSegment
import os
import uuid
import scipy.io.wavfile
import os
import time
import re
from transformers import AutoTokenizer

class TextToSpeechSynthesizer:
    """
    A synthesizer for converting text to speech using a VITS model.
    It outputs the speech in MP3 format.
    """
    def __init__(self, model_name="facebook/mms-tts-kir", output_dir="./media/voices"):
        self.model = VitsModel.from_pretrained(model_name)  
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.output_dir = output_dir
        os.makedirs(self.output_dir, exist_ok=True)
        self.number_words = {
            '0': 'ноль', '1': 'бир', '2': 'эки', '3': 'уч', '4': 'төрт', '5': 'беш', '6': 'алты',
            '7': 'жети', '8': 'сегиз', '9': 'тогуз', '10': 'он', '20': 'жыйырма', '30': 'отуз',
            '40': 'кырк', '50': 'элу', '60': 'алтымыш', '70': 'жетимиш', '80': 'сексен',
            '90': 'токсон', '100': 'жүз', '1000': 'миң', '1000000': 'миллион', '1000000000': 'миллиард'
        }

    def synthesize(self, text):
        inputs = self.tokenizer(self.num_to_text(text), return_tensors="pt")
        inputs['input_ids'] = inputs['input_ids'].long()
        if inputs['input_ids'].size(1) == 0:
            raise ValueError("Tokenized input is empty. Check the input text or tokenizer settings.")
        try:
            with torch.no_grad():
                outputs = self.model(**inputs)
        except RuntimeError as e:
            print("Error during model inference:", e)
            raise
        waveform = outputs.waveform.squeeze().cpu().numpy()
        sampling_rate = self.model.config.sampling_rate
        uid = str(uuid.uuid4())
        mp3_path = os.path.join(self.output_dir, f"{uid}.mp3")
        scipy.io.wavfile.write(mp3_path, rate=sampling_rate, data=waveform)
        time.sleep(3)
        return mp3_path

    def num_to_text(self, text):
    
        def kyrgyz_number(n):
            if n < 10:
                return self.number_words[str(n)]
            elif n < 100:
                tens, below_ten = divmod(n, 10)
                if below_ten == 0:
                    return self.number_words[str(tens * 10)]
                else:
                    return self.number_words[str(tens * 10)] + " " + self.number_words[str(below_ten)]
            elif n < 1000:
                hundreds, below_hundred = divmod(n, 100)
                result = (self.number_words[str(hundreds)] + ' ' if hundreds > 1 else "") + self.number_words['100']
                if below_hundred != 0:
                    result += " " + kyrgyz_number(below_hundred)
                return result
            elif n < 1000000:
                thousands, below_thousand = divmod(n, 1000)
                result = kyrgyz_number(thousands) + ' ' + self.number_words['1000']
                if below_thousand != 0:
                    result += " " + kyrgyz_number(below_thousand)
                return result
            elif n < 1000000000:
                millions, below_million = divmod(n, 1000000)
                result = kyrgyz_number(millions) + ' ' + self.number_words['1000000']
                if below_million != 0:
                    result += " " + kyrgyz_number(below_million)
                return result
            else:
                billions, below_billion = divmod(n, 1000000000)
                result = kyrgyz_number(billions) + ' ' + self.number_words['1000000000']
                if below_billion != 0:
                    result += " " + kyrgyz_number(below_billion)
                return result

        # Replace numbers in text with their

if __name__ == "__main__":
    output_directory = "media/voices"
    synthesizer = TextToSpeechSynthesizer(output_dir=output_directory)
    example_text = "   " + " 500 миң сап ырды түзөт."
    example_text = synthesizer.num_to_text(example_text.strip())
    try:
        mp3_file = synthesizer.synthesize(example_text)
        print(f"Generated speech saved to {mp3_file}")
    except Exception as e:
        print(f"Error: {e}")


Some weights of the model checkpoint at facebook/mms-tts-kir were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

Error: You need to specify either `text` or `text_target`.


In [89]:
import torch
from transformers import VitsModel, AutoTokenizer
import numpy as np
import scipy.io.wavfile
import os
import uuid
import time

class TextToSpeechSynthesizer:
    """
    A synthesizer for converting text to speech using a VITS model.
    It outputs the speech in MP3 format.
    """
    def __init__(self, model_name="facebook/mms-tts-kir", output_dir="./media/voices"):
        self.model = VitsModel.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.output_dir = output_dir
        os.makedirs(self.output_dir, exist_ok=True)
        self.number_words = {
            '0': 'нол', '1': 'бир', '2': 'эки', '3': 'уч', '4': 'төрт', '5': 'беш', '6': 'алты',
            '7': 'жети', '8': 'сегиз', '9': 'тогуз', '10': 'он', '20': 'жыйырма', '30': 'отуз',
            '40': 'кырк', '50': 'элү', '60': 'алтымыш', '70': 'жетимиш', '80': 'сексен',
            '90': 'токсон', '100': 'жүз', '1000': 'миң', '1000000': 'миллион', '1000000000': 'миллиард'
        }

    def synthesize(self, text):
        inputs = self.tokenizer(text, return_tensors="pt")
        inputs['input_ids'] = inputs['input_ids'].long()
        if inputs['input_ids'].size(1) == 0:
            raise ValueError("Tokenized input is empty. Check the input text or tokenizer settings.")
        try:
            with torch.no_grad():
                outputs = self.model(**inputs)
        except RuntimeError as e:
            print("Error during model inference:", e)
            raise
        waveform = outputs.waveform.squeeze().cpu().numpy()
        sampling_rate = self.model.config.sampling_rate
        uid = str(uuid.uuid4())
        mp3_path = os.path.join(self.output_dir, f"{uid}.mp3")
        scipy.io.wavfile.write(mp3_path.replace('.mp3', '.wav'), rate=sampling_rate, data=waveform)
        time.sleep(3)
        return mp3_path

    def kyrgyz_number(self, n):
        if n == 0:
            return self.number_words['0']
        elif n < 10:
            return self.number_words[str(n)]
        elif n < 100:
            tens, below_ten = divmod(n, 10)
            if below_ten == 0:
                return self.number_words[str(tens * 10)]
            else:
                return self.number_words[str(tens * 10)] + " " + self.number_words[str(below_ten)]
        elif n < 1000:
            hundreds, below_hundred = divmod(n, 100)
            result = (self.number_words[str(hundreds)] + ' ' if hundreds > 1 else "") + self.number_words['100']
            if below_hundred != 0:
                result += " " + self.kyrgyz_number(below_hundred)
            return result
        elif n < 1000000:
            thousands, below_thousand = divmod(n, 1000)
            result = self.kyrgyz_number(thousands) + ' ' + self.number_words['1000']
            if below_thousand != 0:
                result += " " + self.kyrgyz_number(below_thousand)
            return result
        elif n < 1000000000:
            millions, below_million = divmod(n, 1000000)
            result = self.kyrgyz_number(millions) + ' ' + self.number_words['1000000']
            if below_million != 0:
                result += " " + self.kyrgyz_number(below_million)
            return result
        else:
            billions, below_billion = divmod(n, 1000000000)
            result = self.kyrgyz_number(billions) + ' ' + self.number_words['1000000000']
            if below_billion != 0:
                result += " " + self.kyrgyz_number(below_billion)
            return result

    def num_to_text(self, text):
        words = text.split()
        result_words = []
        for word in words:
            if word.isdigit():
                num = int(word)
                converted_number = self.kyrgyz_number(num)
                result_words.append(converted_number)
            else:
                result_words.append(word)
        return ''.join(result_words)

if __name__ == "__main__":
    output_directory = "./media/voices"
    synthesizer = TextToSpeechSynthesizer(output_dir=output_directory)
    example_text = "   " + " 500 миң сап ырды түзөт. 200 400 400 3000 645 "
    example_text = synthesizer.num_to_text(example_text.strip())
    try:
        mp3_file = synthesizer.synthesize(example_text)
        print(f"Generated speech saved to {mp3_file}")
    except Exception as e:
        print(f"Error: {e}")


Some weights of the model checkpoint at facebook/mms-tts-kir were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

Generated speech saved to ./media/voices\965704ed-b1cd-4c53-95ce-4f2866b5205e.mp3
